In [1]:
import pandas as pd
import os
import copy
import unittest
import string
import re

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines

In [2]:
lines = read_file("C:/Code/advent/2021_day22_input.txt")

In [3]:
lines_example1 = read_file("C:/Code/advent/2021_day22_example1.txt")
lines_example1

['on x=10..12,y=10..12,z=10..12',
 'on x=11..13,y=11..13,z=11..13',
 'off x=9..11,y=9..11,z=9..11',
 'on x=10..10,y=10..10,z=10..10']

In [4]:
lines_example2 = read_file("C:/Code/advent/2021_day22_example2.txt")
lines_example2

['on x=-20..26,y=-36..17,z=-47..7',
 'on x=-20..33,y=-21..23,z=-26..28',
 'on x=-22..28,y=-29..23,z=-38..16',
 'on x=-46..7,y=-6..46,z=-50..-1',
 'on x=-49..1,y=-3..46,z=-24..28',
 'on x=2..47,y=-22..22,z=-23..27',
 'on x=-27..23,y=-28..26,z=-21..29',
 'on x=-39..5,y=-6..47,z=-3..44',
 'on x=-30..21,y=-8..43,z=-13..34',
 'on x=-22..26,y=-27..20,z=-29..19',
 'off x=-48..-32,y=26..41,z=-47..-37',
 'on x=-12..35,y=6..50,z=-50..-2',
 'off x=-48..-32,y=-32..-16,z=-15..-5',
 'on x=-18..26,y=-33..15,z=-7..46',
 'off x=-40..-22,y=-38..-28,z=23..41',
 'on x=-16..35,y=-41..10,z=-47..6',
 'off x=-32..-23,y=11..30,z=-14..3',
 'on x=-49..-5,y=-3..45,z=-29..18',
 'off x=18..30,y=-20..-8,z=-3..13',
 'on x=-41..9,y=-7..43,z=-33..15',
 'on x=-54112..-39298,y=-85059..-49293,z=-27449..7877',
 'on x=967..23432,y=45373..81175,z=27513..53682']

In [5]:
lines_example3 = read_file("C:/Code/advent/2021_day22_example3.txt")

In [6]:
lines_example4 = read_file("C:/Code/advent/2021_day22_example4.txt")

In [7]:
def parse_range(input):
    parts = input.split('=')[1].split('..')    
    return list(map(int,parts))

def parse_row(line):
    parts = line.split(' ')
    signal = 1 if parts[0] == 'on' else 0
    ranges = list(map(parse_range,parts[1].split(',')))
    return (signal, ranges)


def parse(input):
    result = []
    for line in input:
        result.append(parse_row(line))
    return result

parse(lines_example1)

[(1, [[10, 12], [10, 12], [10, 12]]),
 (1, [[11, 13], [11, 13], [11, 13]]),
 (0, [[9, 11], [9, 11], [9, 11]]),
 (1, [[10, 10], [10, 10], [10, 10]])]

In [8]:
def is_in_ranges(ranges,coordinate):
    for d in range(0,3):
        if not(coordinate[d] >= ranges[d][0] and coordinate[d] <= ranges[d][1]):
            return False
    return True

def is_on(input, coordinate):
    state = 0
    for (signal, ranges) in input:
        if (signal ==1 and state == 0) or (signal == 0 and state == 1):
            if is_in_ranges(ranges, coordinate):
                state = signal        
    return state

def calculate_part_one(input):
    total = 0
    for x in range(-50,51):
        for y in range(-50,51):
            for z in range(-50,51):
                total += is_on(input, (x,y,z))
    return total
    
calculate_part_one(parse(lines_example1))

39

In [8]:
#calculate_part_one(parse(lines_example2))

In [9]:
#calculate_part_one(parse(lines))

In [10]:
#calculate_part_one(parse(lines_example3))

In [35]:
def get_range_map(input, dim):
    result = {}
    coordinates = set()
    for (signal, ranges) in input:
        slice = ranges[dim]
        for x in slice:
            coordinates.add(x)
    coordinates_list = list(coordinates)
    coordinates_list.sort()
    for i in range(0,len(coordinates_list)-1):
        width = coordinates_list[i+1] - coordinates_list[i]
        result[coordinates_list[i]] = 1
        if width > 1:
            result[coordinates_list[i]+1] = width - 1
    result[coordinates_list[len(coordinates_list)-1]] = 1
    return result

def get_range_maps(input):
    result = []
    for d in range(0,3):
        result.append(get_range_map(input, d))
    return result

def calculate_part_two(input):
    range_maps = get_range_maps(input)    
    total = 0
    for x in range_maps[0]:
        print(x)
        for y in range_maps[1]:
            for z in range_maps[2]:
                size = range_maps[0][x] * range_maps[1][y] * range_maps[2][z]
                #print(size, (x, y, z), (range_maps[0][x] , range_maps[1][y] , range_maps[2][z]), is_on(input, (x,y,z)) * size)
                total += is_on(input, (x,y,z)) * size
    print(total)

def filtered_dictionary(range_map, input):
    result = {}
    for key in range_map:
        if key >= input[0] and key <= input[1]:
            result[key] = range_map[key]
    return result

def has_visited(input, point):
    for r in input:
        if is_in_ranges(r, point):
            return True
    return False

def calculate_part_two_alt(input):
    range_maps = get_range_maps(input)    
    total = 0
    visited = []
    visited_count = 0
    for i in range(len(input)):
        print('step', i , visited_count)
        current_signal, current_range = input[i]
        if current_signal == 0:
            continue
        x_range = filtered_dictionary(range_maps[0], current_range[0])
        y_range = filtered_dictionary(range_maps[1], current_range[1])
        z_range = filtered_dictionary(range_maps[2], current_range[2])
        for x in x_range:
            for y in y_range:
                for z in z_range:
                    point = (x,y,z)
                    if has_visited(visited, point):
                        continue
                    visited_count += 1
                    size = range_maps[0][x] * range_maps[1][y] * range_maps[2][z]
                    #print(size, (x, y, z), (range_maps[0][x] , range_maps[1][y] , range_maps[2][z]), is_on(input, (x,y,z)) * size)
                    if is_on(input, (x,y,z)):
                        total += is_on(input, (x,y,z)) * size
                        #print('increased total to', total, visited_count)
        
        #print('visited' , current_range)
        visited.append(current_range)
    print(total)

calculate_part_two_alt(parse(lines_example1))

step 0 0
step 1 27
step 2 46
step 3 46
39


In [44]:
calculate_part_two(parse(lines_example3))

-120100
-120099
-111166
-111165
-110886
-110885
-101086
-101085
-98497
-98496
-98156
-98155
-95822
-95821
-93533
-93532
-89813
-89812
-84341
-84340
-83015
-83014
-77139
-77138
-72682
-72681
-70369
-70368
-65152
-65151
-61695
-61694
-60785
-60784
-57817
-57816
-57795
-57794
-55829
-55828
-53655
-53654
-53470
-53469
-52752
-52751
-47488
-47487
-44271
-44270
-40997
-40996
-37810
-37809
-34664
-34663
-34419
-34418
-32970
-32969
-32178
-32177
-29982
-29981
-27851
-27850
-27365
-27364
-21330
-21329
-18631
-18630
-18186
-18185
-16602
-16601
-16548
-16547
-16478
-16477
-15171
-15170
-14614
-14613
-12347
-12346
-11147
-11146
-9461
-9460
-9439
-9438
-6158
-6157
-5813
-5812
-4958
-4957
-4276
-4275
-1262
-1261
-1060
-1059
-726
-725
-49
-48
-44
-43
-42
-41
-40
-33
-32
-20
-19
-14
-13
-5
-4
-1
0
5
6
15
16
26
27
34
35
36
37
39
40
47
48
2032
2033
8476
8477
10506
10507
11593
11594
12091
12092
13528
13529
13987
13988
14781
14782
16383
16384
17935
17936
18020
18021
18248
18249
21291
21292
22273
22274
249

In [43]:
calculate_part_two_alt(parse(lines_example3))

step 0 0
step 1 6375
step 2 9268
step 3 11622
step 4 14128
step 5 14128
step 6 15582
step 7 15582
step 8 15754
step 9 15754
step 10 17342
step 11 183269
step 12 552407
step 13 744851
step 14 1058753
step 15 1334091
step 16 1525982
step 17 1620167
step 18 2243447
step 19 2759761
step 20 3552761
step 21 3962442
step 22 4529777
step 23 4805818
step 24 4853242
step 25 5394815
step 26 5719075
step 27 6239443
step 28 6333592
step 29 6368851
step 30 6449233
step 31 6719996
step 32 6778043
step 33 7130324
step 34 7165346
step 35 7172006
step 36 7287368
step 37 7510518
step 38 7510518
step 39 7590842
step 40 7691186
step 41 7770803
step 42 7833041
step 43 7833041
step 44 7833041
step 45 7903645
step 46 7903645
step 47 7903645
step 48 7903645
step 49 7907685
step 50 7907685
step 51 7907685
step 52 7907685
step 53 7907685
step 54 7915033
step 55 7915033
step 56 7915033
step 57 7915033
step 58 7915033
step 59 8015205
2758514936282235


In [38]:
def part_one_range(input):
    result = []
    for (signal, ranges) in input:
        add = True
        for r in ranges:
            for x in r:
                if x < -50 or x > 50:
                    add = False
        if add:
            result.append((signal,ranges))
    return result

part_one_range(parse(lines_example3))
calculate_part_two_alt(part_one_range(parse(lines_example3)))

step 0 0
step 1 6375
step 2 9268
step 3 11622
step 4 14128
step 5 14128
step 6 15582
step 7 15582
step 8 15754
step 9 15754
474140


In [40]:
for item in part_one_range(parse(lines_example2)):
    print(item)
print(calculate_part_one(part_one_range(parse(lines_example2))))    
calculate_part_two_alt(part_one_range(parse(lines_example2)))

(1, [[-20, 26], [-36, 17], [-47, 7]])
(1, [[-20, 33], [-21, 23], [-26, 28]])
(1, [[-22, 28], [-29, 23], [-38, 16]])
(1, [[-46, 7], [-6, 46], [-50, -1]])
(1, [[-49, 1], [-3, 46], [-24, 28]])
(1, [[2, 47], [-22, 22], [-23, 27]])
(1, [[-27, 23], [-28, 26], [-21, 29]])
(1, [[-39, 5], [-6, 47], [-3, 44]])
(1, [[-30, 21], [-8, 43], [-13, 34]])
(1, [[-22, 26], [-27, 20], [-29, 19]])
(0, [[-48, -32], [26, 41], [-47, -37]])
(1, [[-12, 35], [6, 50], [-50, -2]])
(0, [[-48, -32], [-32, -16], [-15, -5]])
(1, [[-18, 26], [-33, 15], [-7, 46]])
(0, [[-40, -22], [-38, -28], [23, 41]])
(1, [[-16, 35], [-41, 10], [-47, 6]])
(0, [[-32, -23], [11, 30], [-14, 3]])
(1, [[-49, -5], [-3, 45], [-29, 18]])
(0, [[18, 30], [-20, -8], [-3, 13]])
(1, [[-41, 9], [-7, 43], [-33, 15]])
590784
step 0 0
step 1 24128
step 2 40414
step 3 44690
step 4 61828
step 5 73378
step 6 76697
step 7 80259
step 8 86200
step 9 88064
step 10 88076
step 11 88076
step 12 96904
step 13 96904
step 14 102048
step 15 102048
step 16 108548
ste

In [41]:
print(calculate_part_one(part_one_range(parse(lines_example4))))    
calculate_part_two(part_one_range(parse(lines_example4)))
print(get_range_maps(parse(lines_example4)))

1716
1
2
11
1716
[{1: 1, 2: 9, 11: 1}, {1: 1, 2: 10, 12: 1}, {1: 1, 2: 11, 13: 1}]


In [42]:
calculate_part_two_alt(parse(lines))

step 0 0
step 1 22540
step 2 35000
step 3 43469
step 4 51035
step 5 58214
step 6 69116
step 7 75378
step 8 80322
step 9 87308
step 10 94188
step 11 94188
step 12 94968
step 13 94968
step 14 96194
step 15 96194
step 16 96244
step 17 96244
step 18 96299
step 19 96299
step 20 96425


KeyboardInterrupt: 

In [65]:
def range_1d_intersect(first, second):
    if first[0] < second[0]:
        return second[0] <= first[1]
    else:
        return second[1] >= first[0]

def test_range_1d_intersect(first, second):
    print(first, second, range_1d_intersect(first, second))
    print(second, first, range_1d_intersect(second, first))

test_range_1d_intersect([3,4],[1,6])
test_range_1d_intersect([3,4],[1,2])
test_range_1d_intersect([3,4],[1,3])
test_range_1d_intersect([0,10],[5,20])

[3, 4] [1, 6] True
[1, 6] [3, 4] True
[3, 4] [1, 2] False
[1, 2] [3, 4] False
[3, 4] [1, 3] True
[1, 3] [3, 4] True
[0, 10] [5, 20] True
[5, 20] [0, 10] True


In [69]:
def range_1d_split(first, second):
    if first[0] <= second[0]:
        if second[0] <= first[1]:
            if second[1] <= first[1]:                
                return [[second[0],second[1]]]
            else:
                return [[second[0],first[1]],[first[1]+1,second[1]]]
        else:
            return []
    else:
        if second[1] > first[1]:
            return [[second[0],first[0]-1],[first[0],first[1]],[first[1]+1,second[1]]]
        elif second[1] >= first[0]:
            return [[second[0],first[0]-1],[first[0],second[1]]]
        else:
            return []

def test_range_1d_split(first, second):
    print(first, second, range_1d_split(first, second))
    print(second, first, range_1d_split(second, first))

test_range_1d_split([3,4],[1,6])
test_range_1d_split([3,4],[1,2])
test_range_1d_split([3,4],[1,3])
test_range_1d_split([0,10],[5,20])
test_range_1d_split([0,10],[5,8])

[3, 4] [1, 6] [[1, 2], [3, 4], [5, 6]]
[1, 6] [3, 4] [[3, 4]]
[3, 4] [1, 2] []
[1, 2] [3, 4] []
[3, 4] [1, 3] [[1, 2], [3, 3]]
[1, 3] [3, 4] [[3, 3], [4, 4]]
[0, 10] [5, 20] [[5, 10], [11, 20]]
[5, 20] [0, 10] [[0, 4], [5, 10]]
[0, 10] [5, 8] [[5, 8]]
[5, 8] [0, 10] [[0, 4], [5, 8], [9, 10]]


In [78]:
def has_intersection(first, second):
    for d in range(0,3):
        if not range_1d_intersect(first[d], second[d]):
            return False
    return True

def break_into_parts(first, second):
    result = []
    x_ranges = range_1d_split(first[0],second[0])
    y_ranges = range_1d_split(first[1],second[1])
    z_ranges = range_1d_split(first[2],second[2])
    for x_range in x_ranges:
        for y_range in y_ranges:
            for z_range in z_ranges:
                result.append([x_range, y_range, z_range])
    return result

def handle(input, cubes):
    print(input, len(cubes))   
    signal, ranges = input    
    if signal:
        result = cubes
        remaining = [ranges]
        while len(remaining) > 0:
            current = remaining.pop()
            can_add = True
            for c in cubes:
                if has_intersection(current, c):
                    can_add = False
                    subcubes = break_into_parts(c, current)
                    #print('break subcubes', c, current, subcubes )
                    for subcube in subcubes:
                        if not has_intersection(subcube, c):
                            #print('adding subcube', subcube)
                            remaining.append(subcube)
                    break
            if can_add:
                #print('appending',current)
                cubes.append(current)
        return result
    else:
        # got an eraser...
        result = []
        eraser = ranges
        for c in cubes:
            if has_intersection(c, eraser):
                subcubes = break_into_parts(eraser, c)
                for subcube in subcubes:
                    if not has_intersection(eraser, subcube):
                        result.append(subcube)
            else:
                result.append(c)
        return result

def reduce_to_lights(cubes):
    total = 0
    for c in cubes:
        volume = 1
        for dim in range(0,3):
            volume *= (c[dim][1]-c[dim][0] + 1)
        total += volume
    return total


def calculate_part_two_merge(input):
    cubes = []
    n = 0
    for i in input:
        n+=1
        print('step',n , '-------', i, i[0])
        cubes = handle(i, cubes)
        print('cubes', len(cubes))
    return reduce_to_lights(cubes)

calculate_part_two_merge(parse(lines_example1))

step 1 ------- (1, [[10, 12], [10, 12], [10, 12]]) 1
(1, [[10, 12], [10, 12], [10, 12]]) 0
cubes 1
step 2 ------- (1, [[11, 13], [11, 13], [11, 13]]) 1
(1, [[11, 13], [11, 13], [11, 13]]) 1
cubes 8
step 3 ------- (0, [[9, 11], [9, 11], [9, 11]]) 0
(0, [[9, 11], [9, 11], [9, 11]]) 8
cubes 14
step 4 ------- (1, [[10, 10], [10, 10], [10, 10]]) 1
(1, [[10, 10], [10, 10], [10, 10]]) 14
cubes 15


39

In [79]:
calculate_part_two_merge(part_one_range(parse(lines_example2)))

step 1 ------- (1, [[-20, 26], [-36, 17], [-47, 7]]) 1
(1, [[-20, 26], [-36, 17], [-47, 7]]) 0
cubes 1
step 2 ------- (1, [[-20, 33], [-21, 23], [-26, 28]]) 1
(1, [[-20, 33], [-21, 23], [-26, 28]]) 1
cubes 8
step 3 ------- (1, [[-22, 28], [-29, 23], [-38, 16]]) 1
(1, [[-22, 28], [-29, 23], [-38, 16]]) 8
cubes 19
step 4 ------- (1, [[-46, 7], [-6, 46], [-50, -1]]) 1
(1, [[-46, 7], [-6, 46], [-50, -1]]) 19
cubes 36
step 5 ------- (1, [[-49, 1], [-3, 46], [-24, 28]]) 1
(1, [[-49, 1], [-3, 46], [-24, 28]]) 36
cubes 58
step 6 ------- (1, [[2, 47], [-22, 22], [-23, 27]]) 1
(1, [[2, 47], [-22, 22], [-23, 27]]) 58
cubes 69
step 7 ------- (1, [[-27, 23], [-28, 26], [-21, 29]]) 1
(1, [[-27, 23], [-28, 26], [-21, 29]]) 69
cubes 98
step 8 ------- (1, [[-39, 5], [-6, 47], [-3, 44]]) 1
(1, [[-39, 5], [-6, 47], [-3, 44]]) 98
cubes 144
step 9 ------- (1, [[-30, 21], [-8, 43], [-13, 34]]) 1
(1, [[-30, 21], [-8, 43], [-13, 34]]) 144
cubes 163
step 10 ------- (1, [[-22, 26], [-27, 20], [-29, 19]]) 1
(1, 

590784

In [80]:
calculate_part_two_merge(parse(lines_example3))

step 1 ------- (1, [[-5, 47], [-31, 22], [-19, 33]]) 1
(1, [[-5, 47], [-31, 22], [-19, 33]]) 0
cubes 1
step 2 ------- (1, [[-44, 5], [-27, 21], [-14, 35]]) 1
(1, [[-44, 5], [-27, 21], [-14, 35]]) 1
cubes 4
step 3 ------- (1, [[-49, -1], [-11, 42], [-10, 38]]) 1
(1, [[-49, -1], [-11, 42], [-10, 38]]) 4
cubes 21
step 4 ------- (1, [[-20, 34], [-40, 6], [-44, 1]]) 1
(1, [[-20, 34], [-40, 6], [-44, 1]]) 21
cubes 30
step 5 ------- (0, [[26, 39], [40, 50], [-2, 11]]) 0
(0, [[26, 39], [40, 50], [-2, 11]]) 30
cubes 30
step 6 ------- (1, [[-41, 5], [-41, 6], [-36, 8]]) 1
(1, [[-41, 5], [-41, 6], [-36, 8]]) 30
cubes 50
step 7 ------- (0, [[-43, -33], [-45, -28], [7, 25]]) 0
(0, [[-43, -33], [-45, -28], [7, 25]]) 50
cubes 56
step 8 ------- (1, [[-33, 15], [-32, 19], [-34, 11]]) 1
(1, [[-33, 15], [-32, 19], [-34, 11]]) 56
cubes 70
step 9 ------- (0, [[35, 47], [-46, -34], [-11, 5]]) 0
(0, [[35, 47], [-46, -34], [-11, 5]]) 70
cubes 70
step 10 ------- (1, [[-14, 36], [-6, 44], [-16, 29]]) 1
(1, [[-1

2758514936282235

In [81]:
calculate_part_two_merge(parse(lines))

step 1 ------- (1, [[-30, 22], [-28, 20], [-17, 37]]) 1
(1, [[-30, 22], [-28, 20], [-17, 37]]) 0
cubes 1
step 2 ------- (1, [[-20, 34], [-30, 17], [-30, 21]]) 1
(1, [[-20, 34], [-30, 17], [-30, 21]]) 1
cubes 8
step 3 ------- (1, [[-7, 42], [-23, 26], [-18, 29]]) 1
(1, [[-7, 42], [-23, 26], [-18, 29]]) 8
cubes 23
step 4 ------- (1, [[-8, 44], [-34, 20], [-14, 32]]) 1
(1, [[-8, 44], [-34, 20], [-14, 32]]) 23
cubes 51
step 5 ------- (1, [[-13, 32], [-47, 6], [-38, 6]]) 1
(1, [[-13, 32], [-47, 6], [-38, 6]]) 51
cubes 69
step 6 ------- (1, [[-48, 6], [-36, 14], [-18, 33]]) 1
(1, [[-48, 6], [-36, 14], [-18, 33]]) 69
cubes 100
step 7 ------- (1, [[-31, 13], [-22, 32], [-44, 9]]) 1
(1, [[-31, 13], [-22, 32], [-44, 9]]) 100
cubes 133
step 8 ------- (1, [[-18, 31], [-48, -2], [-18, 28]]) 1
(1, [[-18, 31], [-48, -2], [-18, 28]]) 133
cubes 162
step 9 ------- (1, [[-9, 38], [-15, 35], [-48, 6]]) 1
(1, [[-9, 38], [-15, 35], [-48, 6]]) 162
cubes 212
step 10 ------- (1, [[-14, 35], [-42, 6], [-9, 43]]

1255547543528356